### ETL Steam Games

Este Archivo de trabajo, contiene las instrucciones necesarias para ejecutar la extración, transformación y carga del archivo "steam_games.json".

La intención inicial del documento, es mostrar el paso a paso de la extracción de la información, explicar y justificar cada una de las instrucciones ejecutadas para la transformación del archivo y brindar como resultado un archivo limpio en formato CSV.

In [1]:
# Se importan las librerías necesarias para trabajar con archivos de tipo json
import pandas as pd # Se utiliza pandas para trabajar con dataframes
import json # Se utiliza json para la lectura de archivos en este formato
import numpy as np # Se utiliza para operaciones matematicas dentro del dataframe

In [2]:
# Se comienza con la lectura del archivo json y colocar su contenido en un dataframe de pandas
Ruta_Json = '..\..\Inputs Originales\output_steam_games.json' # Guardamos la ruta del archivo json a leer en una variable

# Procedemos a leer el archivo json
with open(Ruta_Json, 'r') as archivo: # Se lee el archivo json utilizando la variable json
    # Debido a que cada línea de nuestro archivo es un objeto de tipo json, se procede a leer cada línea del archivo y se guarda en una lista general
    Contenido = [json.loads(linea) for linea in archivo] # Se carga en una lista cada línea leída del archivo

df_Steam_Games = pd.DataFrame(Contenido) # Se convierte en data frame el resultado de la lectura del archivo json

In [3]:
# Ahora que tenemos un Dataframe con la información del archivo, comenzamos a explorar el contenido y realizamos ciertos trabajos de limpieza

# Analizamos la estructura y contenido del dataframe
df_Steam_Games.info() # Nos muestra la información general del dataframe
# Por ahora, tenemos un total de 13 columnas de tipo objeto

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [4]:
# haciendo un análisis estadístico del Dataframe, veremos cuántos registros contiene, el total de vacíos y duplicados

print("Total de filas en el DataFrame: ", df_Steam_Games.shape[0]) # Mostramos el total de registros del DataFrame 
# Se tiene un total de 120,445 registros dentro del archivo
print("Total de registros vacíos por columna: ", df_Steam_Games.isnull().sum()) # Mostramos el total de registros vacíos

# Se reemplazan los valores vacíos descritos como 'null', 'None' con NaN en todo el DataFrame
df_Steam_Games.replace(['', 'null', 'None'], np.nan, inplace=True) # Se guardel resultaddo en el DataFrame
# Se procede a eliminar aquellos registros que estén vacíos
df_Steam_Games.dropna(inplace = True) # Se guarda el resultado en el mismo dataframe
print("Total de filas en el DataFrame sin registros vacíos: ", df_Steam_Games.shape[0]) # Mostramos el total de registros del DataFrame 
# Se tiene un total de 22,521 registros dentro del archivo
# Con esta eliminación, se da a notar que aproximadamente el 81% del contenido del archivo eran registros vacíos

Total de filas en el DataFrame:  120445
Total de registros vacíos por columna:  publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64
Total de filas en el DataFrame sin registros vacíos:  22521


In [5]:
# Seguimos analizando de forma general la estadística de cada columna
df_Steam_Games.describe() # Se muestra la descripción general del DataFrame
# Se muestra que hay un ID repetido, después de analizarlo, nos damos cuenta que hay que eliminar el primer registro duplicado
df_Steam_Games = df_Steam_Games.drop_duplicates(subset='id', keep='last') # Eliminamos el primer registro duplicado

In [6]:
# Modificamos el orden de las columnas, para establecer el Id como el índice del dataframe y actualizamos su valor
df_Steam_Games.rename(columns={'app_name': 'name', 'id': 'item_id'}, inplace=True) # Renombramos dos columnas
df_Steam_Games = df_Steam_Games[["item_id"] + [col for col in df_Steam_Games.columns if col != "item_id"]] # Se reorganizan las columnas, para poder la columna "item_id" en primera posición
df_Steam_Games.sort_index(inplace = True) # Se ordena el dataframe por su ID de forma ascendente

In [7]:
# Se convierte la columna 'genres' a un tipo lista y no string
# Se aplica función lambda para aplicar las condiciones a cada fila
df_Steam_Games['genres'] = df_Steam_Games['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# Se convierte la columna "tags" a un tipo lista y no un string
df_Steam_Games['tags'] = df_Steam_Games['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Se expanden los valores obtenidos en las columnas genres, tags y specs y se guardan en un nuevo dataframe
df_Steam_Games = df_Steam_Games.explode('genres') # Se expanden los valores de genres
df_Steam_Games = df_Steam_Games.explode('tags') # Se expanden los valores de tags
df_Steam_Games = df_Steam_Games.explode('specs') # Se expanden los valores de specs

In [8]:
# Con la finalidad de hacer más optimo nuestro dataframe, se eliminan aquellas columnas que no se consideran relevantes para el análisis
df_Steam_Games = df_Steam_Games.drop(['title','url','reviews_url','price','early_access'], axis=1) # Se procede a eliminar aquellas columnas que no serían útiles para el análisis o que se pueden obtener desde otro csv
df_Steam_Games = df_Steam_Games.drop_duplicates(keep='first') # Se liminan duplicados del dataframe
df_Steam_Games.dropna(inplace = True) # Se eliminan los registros vacíos

In [9]:
# Guardamos nuestro Dataframe final en un archivo de tipo CSV Limpio
df_Steam_Games.to_csv('..\..\Archivos Limpios\output_steam_games_limpio.csv', index=False)

# Por ahora, se ha terminado con la limpieza de la información del archivo steam games